# Метод 3
## Использование линейных моделей

#### У нас уже есть словарь с названиями категорий и словами

In [1]:
import pandas as pd

In [2]:
data = pd.read_pickle("res.p")

In [3]:
for key in data:
    len([a for a in data[key] if a])

85

24

91

98

93

...

90

93

91

90

86

In [4]:
X, Y = [], []
k = 0
for key in data:
    value = data[key]
    for words in value:
        if words:
            X.append(" ".join(words))
            Y.append(key)
        else:
            k += 1

In [5]:
len(X), len(Y), k

(1024, 1024, 66)

### Снова сделаем из этого векторы и обучим модель
#### Сразу напишем Pipeline для этого

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.cross_validation import train_test_split

from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

text_clf_1 = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('classifier', SGDClassifier()),
])

text_clf_2 = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('classifier', RandomForestClassifier())
])

In [8]:
text_clf_1.fit(X_train, Y_train)
text_clf_2.fit(X_train, Y_train)

Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...   penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False))])

Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [9]:
predicted_1 = text_clf_1.predict(X_test)
predicted_2 = text_clf_2.predict(X_test)

In [10]:
import numpy as np

In [11]:
np.mean(predicted_1 == Y_test)
np.mean(predicted_2 == Y_test) 

0.72682926829268291

0.58048780487804874

# Да уж, что-то RandomForestClassifier не оправдал моих ожиданий на стандартных настройках. Оставим его и продолжим работать с SGDClassifier

In [12]:
predicted_1[36]

'Торговля'

In [13]:
X_test[36][:300]

'мас дилер москва варшавский шоссе карта проездаобратный звонокэкспресс кредитподбор новое авто контакт сколько готовый потратить новый автомобиль рубль рубль рубль рубль рубль рубль рубль рубль рубль рубль рубль рубль рубль рубль рубль рубль рубль рубль рубль рубль рубль рубль рубль рубль рубль рубл'

## 72% просто стандартной конфигурацией...
### Запустим Grid Search, чтобы получить более лучшую конфигурацию

In [14]:
from sklearn.grid_search import GridSearchCV

# Я просто взял параметры отсюда http://scikit-learn.org/dev/auto_examples/model_selection/grid_search_text_feature_extraction.html
# но после первого применения изменил под себя
parameters = {
    'vect__max_df': (0.5, 1.0),
    'vect__max_features': (None, 5000),
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
#     'tfidf__use_idf': (True, False),
#     'tfidf__norm': ('l1', 'l2'),
    'classifier__loss': ('hinge', 'log', 'squared_hinge'),
    'classifier__alpha': (1e-03, 1e-07),
#     'classifier__penalty': ('l2', 'elasticnet'),
    #'clf__n_iter': (10, 50, 80),
}


In [15]:
grid_search = GridSearchCV(text_clf_1, parameters, n_jobs=-1, verbose=1)

In [21]:
grid_search.fit(X_train, Y_train)

Fitting 3 folds for each of 48 candidates, totalling 144 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   27.8s
[Parallel(n_jobs=-1)]: Done 144 out of 144 | elapsed:  1.6min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...   penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'vect__max_df': (0.5, 1.0), 'classifier__alpha': (0.001, 1e-07), 'classifier__loss': ('hinge', 'log', 'squared_hinge'), 'vect__max_features': (None, 5000), 'vect__ngram_range': ((1, 1), (1, 2))},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=1)

In [22]:
print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Best score: 0.751
Best parameters set:
	classifier__alpha: 0.001
	classifier__loss: 'log'
	vect__max_df: 0.5
	vect__max_features: 5000
	vect__ngram_range: (1, 2)


In [16]:
text_clf = Pipeline([('vect', CountVectorizer(max_df=0.5, max_features=5000, ngram_range=(1, 2))),
                    ('tfidf', TfidfTransformer()),
                    ('classifier', SGDClassifier(alpha=0.001, loss='log', penalty='l2')),
])

In [17]:
text_clf.fit(X_train, Y_train)
predicted = text_clf.predict(X_test)
np.mean(predicted == Y_test)

Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.5, max_features=5000, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip...   penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False))])

0.75121951219512195

# Результаты разные. Дадим SGDClassifier'у второй шанс, увеличив и изменив обучающую выборку

In [18]:
data = pd.read_pickle("res_2.p")

In [19]:
for key in data:
    len([a for a in data[key] if a])

106

244

100

25

137

...

228

231

225

233

225

In [132]:
from nltk.corpus import stopwords

stoplist = stopwords.words('russian')

X, Y = [], []
k = 0
for key in data:
    value = data[key]
    for words in value:
        if words:
            # Я вернулся из будущего и понял, что я все-таки хочу удалять предлоги и пр.
            X.append(" ".join([a for a in words if a not in stoplist]))
            Y.append(key)
        else:
            k += 1

In [133]:
len(X), len(Y), k

(2201, 2201, 135)

In [134]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [23]:
text_clf = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('classifier', SGDClassifier()),
])

In [50]:
from sklearn.grid_search import GridSearchCV

parameters = {
    'vect__max_df': (0.5, 1.0),
    'vect__max_features': (None, 5000),
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
#     'tfidf__use_idf': (True, False),
#     'tfidf__norm': ('l1', 'l2'),
    'classifier__loss': ('hinge', 'log', 'squared_hinge'),
    'classifier__alpha': (1e-03, 1e-07),
#     'classifier__penalty': ('l2', 'elasticnet'),
    #'clf__n_iter': (10, 50, 80),
}


In [51]:
grid_search = GridSearchCV(text_clf, parameters, n_jobs=-1, verbose=1)

In [52]:
grid_search.fit(X_train, Y_train)

Fitting 3 folds for each of 48 candidates, totalling 144 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 144 out of 144 | elapsed:  4.4min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...   penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'vect__max_df': (0.5, 1.0), 'classifier__loss': ('hinge', 'log', 'squared_hinge'), 'vect__max_features': (None, 5000), 'vect__ngram_range': ((1, 1), (1, 2)), 'classifier__alpha': (0.001, 1e-07)},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=1)

In [53]:
print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Best score: 0.736
Best parameters set:
	classifier__alpha: 0.001
	classifier__loss: 'hinge'
	vect__max_df: 0.5
	vect__max_features: None
	vect__ngram_range: (1, 1)


In [124]:
# text_clf = Pipeline([('vect', CountVectorizer(max_df=0.5, max_features=None, ngram_range=(1, 1))),
#                     ('tfidf', TfidfTransformer(use_idf=True)),
#                     ('classifier', SGDClassifier(loss='hinge', alpha=0.001)),
# ])

In [128]:
count_vect = CountVectorizer(max_df=0.5, max_features=None, ngram_range=(1, 1))
tfidf_tr = TfidfTransformer(use_idf=True)
sgd_classifier = SGDClassifier(loss='hinge', alpha=0.001)

In [135]:
X_0 = count_vect.fit_transform(X_train)
X_train_0 = tfidf_tr.fit_transform(X_0)
# Y = count_vect.fit_transform(Y_train)
# Y_train = tfidf_tr.fit_transform(Y)

X_0 = count_vect.transform(X_test)
X_test_0 = tfidf_tr.transform(X_0)
sgd_classifier.fit(X_train_0, Y_train)
predicted = sgd_classifier.predict(X_test_0)

# text_clf.fit(X_train, Y_train)
# predicted = text_clf.predict(X_test)
np.mean(predicted == Y_test)

SGDClassifier(alpha=0.001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

0.74376417233560088

In [142]:
import requests
import pymystem3
import re
import numpy as np
from bs4 import BeautifulSoup
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

stopwords = stopwords.words("russian")
mystem = pymystem3.Mystem()
regexp = RegexpTokenizer('[а-яА-Я_]{2,25}')

def method3_0(web_url, count_vect, tfidf_tr, sgd_classifier):
    if re.match("^(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)*\/?$", web_url):
        raw = " ".join(BeautifulSoup(requests.get(web_url).content, "lxml").text.split())
        words = []
        for b in mystem.lemmatize(" ".join([a.lower() for a in regexp.tokenize(raw)])):
             if b != " " and b not in stoplist:
                    words.append(b)
        
        
        X_0 = count_vect.transform([" ".join(words)])
        X_test_0 = tfidf_tr.transform(X_0)
        
        
        return sgd_classifier.predict(X_test_0)
    else:
        print("Возможно, вы предоставили неправильную ссылку")
        return None


In [145]:
t = """/Наука и образование
    /Наука и образование/наука
    /Наука и образование/наука/математика
    /Наука и образование/наука/физика
    /Наука и образование/наука/химия
    /Наука и образование/наука/информатика
    /Наука и образование/наука/информатика/биоинформатика
    /Наука и образование/наука/информатика/анализ данных
    /Наука и образование/наука/литература
    /Наука и образование/образование
    /Наука и образование/образование/школьное
    /Наука и образование/образование/высшее
    /Наука и образование/образование/дополнительное
    /Наука и образование/образование/дополнительное/GoTo
    /Политика
    /Политика/Внутренняя
    /Политика/Внешняя
    /Экономика и бизнес
    /Экономика и бизнес/Бизнес
    /Экономика и бизнес/Бизнес/Стартапы
    /Экономика и бизнес/Бизнес/Стартапы/E-Contenta
    /Экономика и бизнес/Бизнес/Крупные компании
    /Экономика и бизнес/Экономика
    /Отдых и развлечения
    /Отдых и развлечения/Кино
    /Отдых и развлечения/Театр
    /Отдых и развлечения/Компьютерные игры
    /Здоровье и красота/Фитнес
    /Здоровье и красота/Медицина
    /Здоровье и красота/Косметология""".split("\n")

# массив последних тем в иерархии (топики) и словарь, с пом. которого можно восстановить полную структуру
topics = [x[1:].split("/")[-1] for x in t]
restore = {topics[i]: t[i] for i in range(len(topics))}

In [158]:
def method3(web_url):
    try:
        key = method3_0(web_url, count_vect, tfidf_tr, sgd_classifier)[0]
        return restore.get(key.lower(), key)
    except: pass

In [159]:
method3("https://ru.wikipedia.org/wiki/Определитель_Вандермонда")

'    /Наука и образование/образование'

In [160]:
method3("http://goto.msk.ru/hackathon/")

'    /Наука и образование/образование'

In [161]:
method3("https://www.kinopoisk.ru/film/648440/")

'Кино'

In [162]:
method3("https://e-contenta.com/ru/")

'Компьютеры_и_интернет'

In [163]:
method3("https://ru.wikipedia.org/wiki/C%2B%2B")

Возможно, вы предоставили неправильную ссылку
